In [3]:
import json
import pandas as pd
import scipy
import sklearn.preprocessing
import numpy as np
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

In [4]:
events = pd.read_parquet("./data/events.parquet")
items = pd.read_parquet("./data/items.parquet")
user_item_matrix_train = sparse.load_npz("./data/user_item_matrix_train.npz")

In [6]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

In [13]:
item_encoder = LabelEncoder()
item_encoder.fit(items["item_id"])

LabelEncoder()

In [14]:
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])

/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_90717/3741860594.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])


In [8]:
als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/Users/tzepart/anaconda3/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 11 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки implicit, у которого на такой случай есть удобный метод similar_items (подробнее о нём вы можете прочитать в официальной документации).

Код ниже, чтобы получить набор похожих объектов в similar_items. 

In [80]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

In [81]:
print(train_item_ids_enc)

[ 2460 38691 38867 ... 43151 11649 38365]


In [82]:
print(sim_item_item_ids_enc.tolist())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [83]:
print(sim_item_scores.tolist())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [84]:
similar_items = pd.DataFrame(columns=["item_id_enc", "sim_item_id_enc", "score"])

# Convert columns to the appropriate data types
similar_items["score"] = similar_items["score"].astype(float)
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype(int)

similar_items_list = []
for item_id, sim_item_ids, sim_scores in zip(train_item_ids_enc, sim_item_item_ids_enc.tolist(), sim_item_scores.tolist()):
    similar_items_list.append((item_id, sim_item_ids, sim_scores))

# Create DataFrame from the list
similar_items = pd.DataFrame(similar_items_list, columns=["item_id_enc", "sim_item_id_enc", "score"])

In [85]:
# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])

In [86]:
# получаем изначальные идентификаторы
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])

ValueError: y contains previously unseen labels: [list([0, 1, 8250, 4, 3, 1942, 2, 26800, 23781, 18446, 18554])
 list([1, 0, 4, 8250, 3, 1942, 2, 26800, 23781, 18446, 26693])
 list([2, 1942, 3, 4, 1, 0, 8250, 26800, 23781, 26325, 26693]) ...
 list([43232, 42486, 39678, 40246, 41975, 42838, 41425, 40047, 42615, 39624, 31834])
 list([43250, 25004, 27429, 27083, 32656, 26882, 21693, 29549, 32424, 29692, 26959])
 list([43304, 19688, 40743, 40275, 39994, 33817, 37093, 35940, 29855, 31103, 29903])]

In [87]:
# убираем ненужные колонки
# similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])
similar_items = similar_items.drop(columns=["item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [88]:
# Выводим результат
print(similar_items.head())

                                     sim_item_id_enc  \
0  [2460, 2458, 806, 2459, 12528, 1147, 7852, 618...   
1  [38691, 39575, 40111, 25112, 32177, 34430, 367...   
2  [38867, 38023, 38951, 5992, 3865, 10539, 28584...   
3  [39109, 37674, 39384, 40645, 17054, 36002, 394...   
4  [35638, 37837, 41337, 39997, 31205, 25389, 324...   

                                               score  item_id_1  
0  [1.0000001192092896, 0.9224867820739746, 0.874...      22034  
1  [1.0, 0.9343428611755371, 0.9306214451789856, ...   22318578  
2  [0.9999998807907104, 0.9388764500617981, 0.934...   22551730  
3  [1.0, 0.9593720436096191, 0.9475102424621582, ...   22816087  
4  [0.9999997615814209, 0.9470827579498291, 0.944...   17910054  


In [96]:
print(similar_items[similar_items["item_id_1"] == 7126].index)

Index([2352], dtype='int64')


In [100]:
similar_items.to_parquet("./data/similar_items.parquet")

Полезно убедиться, что полученный набор действительно содержит похожие данные. Например, можно оценить глазами списки похожих объектов для каких-то уже известных. Создадим для этой цели функцию print_sim_items.

In [98]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [99]:
print_sim_items(7144, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1909078,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,"{'Classics': 15812, 'Fiction': 8028, 'Cultural...",4.19,390293


KeyError: 'item_id_2'